# Analyzing Models with Ultranest

Now that we have climate models with grids, we need to take it to the next level of analysis with nested sampling utilizing [Ultranest](https://johannesbuchner.github.io/UltraNest/readme.html). This is important as to have a deeper and higher level of statistical support in your analysis of your data.


In this tutorial you will learn how to create/read spectral, chemical, thermal plots and save your results to an [Xarray](https://docs.xarray.dev/en/stable/) for ease of accessibility of use in the future.


In [1]:
import numpy as np
from bokeh.plotting import figure, show
import matplotlib.pyplot as plt
%matplotlib inline

#picaso
import picaso.justplotit as jpi
import picaso.analyze as lyz
jpi.output_notebook()

import pickle as pk

#ULTRANEST
import ultranest
import ultranest.plot as uplt
import ultranest.integrator as uint
from ultranest.plot import PredictionBand

import xarray as xr
import zipfile
import os

Loading BokehJS ...

In [3]:
from model_setup import *

Total number of models in grid is 371
For tint in planet_params grid is: ['200.0' '300.0']
For heat_redis in planet_params grid is: ['0.5' '0.6' '0.7' '0.8']
For mh in planet_params grid is: ['0.0' '0.3' '0.5' '0.7' '1.0' '1.3' '1.5' '1.7' '2.0']
For cto in planet_params grid is: ['0.25' '0.5' '1.0' '1.5' '2.0' '2.5']


# Reading the Results

Now that we have run Ultranest, you should have an `emissions` folder with all of the Ultranest results.
Let's read it and see what it gave us



In [5]:
dirr = 'gridtrieval_files'
res = uint.read_file(dirr, len(params))
res[0].keys()

NameError: name 'params' is not defined

In [8]:
res[1].keys()

dict_keys(['niter', 'logz', 'logzerr', 'logz_bs', 'logz_single', 'logzerr_tail', 'logzerr_bs', 'ess', 'H', 'Herr', 'posterior', 'weighted_samples', 'samples', 'maximum_likelihood', 'insertion_order_MWW_test'])

We can see that it gave us a lot of statistical data which is what we want! There is a couple we really want to know:

`logl`
- Array of log likelihood at each point

`paramnames`
- Parameters we declared in run_ultranest.py

`samples`
- Sampled points from Ultranest

Let's save our parameters/results and create our corner plot. This corner plot will show us the probability distributions of each parameter. In other words, it will show where the parameter has the highestmatching the datalclimate.




## Corner Plot

In [9]:
res[1]['paramnames'] =  params
results = res[1]
paramnames = results['paramnames']
samples = results['samples']
logl = np.array(res[0]['logl'])

uplt.cornerplot(results)

ValueError: need at least one array to stack

In [9]:
summary = jdi.pd.read_csv(os.path.join(dirr, 'info','post_summary.csv'))
median_val = [summary[i+'_'+'median'].values[0] for i in params]
summary

,tint_mean,tint_stdev,tint_median,tint_errlo,tint_errup,heat_redis_mean,heat_redis_stdev,heat_redis_median,heat_redis_errlo,heat_redis_errup,mh_mean,mh_stdev,mh_median,mh_errlo,mh_errup,cto_mean,cto_stdev,cto_median,cto_errlo,cto_errup
0,299.997811,0.00171,299.998213,299.996093,299.99946,0.502143,0.00159,0.501832,0.500562,0.503811,1.997757,0.001689,1.998114,1.995981,1.999404,2.497811,0.001678,2.498122,2.496127,2.499456


# Deciding the best fit physical parameters

There are two paths we can take. 
1) Find the parameter values where the max log likelihood lie.
2) Find the parameter values where the median values lie.

It is really important to take the right path and report what you use. We already have the median values from the corner plot above. In this specific case, let's begin by printing the max log likelfirstod ves.




In [10]:
#Print max log likelihood parameters
for i ,ip in enumerate(paramnames): 
    print(ip,samples[np.argmax(logl),i])
maxlog_val = samples[np.argmax(logl),:]

tint 299.99756358470484
heat_redis 0.5038607196993385
mh 1.9955581068046737
cto 2.499288249359772


Uh oh! Running this twice produces different results. Here are two runs:
> tint 242.90576849771628
>
> heat_redis 0.614576599525345
> 
> mh 1.0190034348328623
>
> cto 0.6516417142035135

vs

> tint 345.25805285732406
>
> heat_redis 0.5943213159419377
>
> mh 1.162968230864151
>
> cto 0.4563875323149711
> 

This means it is much more reliable to use and report the median value parameters, rather than the changing log likelihood values. Let's use the median values for the climate model.

In [ ]:
climate_xarr_dir = "xarrays/climate/hat-p-26_tint300_rfacv0.5_mh+130_cto100.nc" #note we use tint: 300, r_facv/heat_redis: 0.5, mh:1.30, CtoO: 1.0
MODEL = getattr(model_set,model_type)
resultx, resulty = MODEL(median_val)

In [ ]:
#Grab the climate
xr_usr=jdi.xr.load_dataset(climate_xarr_dir)
opa = jdi.opannection(filename_db=opa_dir)
case = jdi.input_xarray(xr_usr, opa)

#copy over the climate
og_atmo = jdi.copy.deepcopy(case.inputs['atmosphere']['profile'])

Let's check the chi-square fit

## Chi-Square Fit

In [ ]:
xd=[]
yd=[]
datay=[]
datae=[]
for i in data_dict.keys(): 
    datay+=list(data_dict[i][1])
    datae+=list(data_dict[i][2])
    xd1, yd1=jdi.mean_regrid(resultx, resulty, newx=data_dict[i][0])
    xd+=list(xd1)
    yd+=list(yd1)

chisq_best_fit_perdata = lyz.chi_squared(np.array(datay),np.array(datae),
                np.array(yd),0)

print(chisq_best_fit_perdata)

# Spectral Anaylsis

It is important to check if the spectral data points from Ultranest align with our actual data.

To begin, let's plot the spectrum data with our sampled results

## Spectrum Data

In [ ]:
x_200, y_200=jdi.mean_regrid(resultx, resulty, R=200)
fig = figure(x_axis_type='log', width=700, height=300)
for i in data_dict.keys():
  
    jpi.plot_errorbar(1e4/data_dict[i][0], 
                  data_dict[i][1], 
                  data_dict[i][2], fig, 
                point_kwargs={'color':'black','size':10, 'legend_label':"Observational Data"},
                      error_kwargs={'color':'black','line_width':4})

fig.line(1e4/x_200, y_200,line_width=4,legend_label="Sampled Points")
fig.legend.location = "top_left"

jpi.show(fig)

Now, let's see the eclipse depth. We are first going to create a band from our observation data.

## Eclipse Depth

In [ ]:
#setting up the band and the error region
MODEL = getattr(model_set,model_type)

def model_w_regrid(eval_at):
    resultx, resulty = MODEL(eval_at)
    x, y=jdi.mean_regrid(resultx, resulty, newx=x_200)
    return y
    


n_draws = 200
samples = results['samples']
draws=np.random.randint(0, samples.shape[0], n_draws)

band = PredictionBand(1e4/x_200)
for idraw in draws:
    band.add(model_w_regrid(samples[idraw,:]))


Next, let's overplot our observation data band with the Ultranest sampled points.

In [ ]:
PuBuGn = jpi.pals.PuBuGn3
fig, ax = plt.subplots(figsize=(15, 7))
 
for i in data_dict.keys():
    ax.errorbar(x=1e4/data_dict[i][0], y=data_dict[i][1], 
             yerr=data_dict[i][2], 
             marker='x', ls=' ')
    
band.line(color='k')
spec_sigmas_hi = {}
spec_sigmas_lo = {}

for q ,c,key in zip([k/100/2 for k in [68.27, 95.45, 99.73]], PuBuGn, ['1sig','2sig','3sig']): 
    band.shade(q=q, color=c, alpha=0.5)
    spec_sigmas_lo[key] = band.get_line(0.5 - q)
    spec_sigmas_hi[key] = band.get_line(0.5 + q)

spec_median_best_fit = band.get_line(0.5)

ax.set_ylabel('Eclipse Depth', fontsize=16)
ax.set_xlabel('Wavelength [$\mu$m]', fontsize=16)

ax.tick_params(axis='x', labelsize=13)
ax.tick_params(axis='y', labelsize=13)
ax.set_xscale('log')
ax.set_xlim(3,12)


Great! It seems like our sampled points align well with the data.

# Pressure-Temperature Analysis

With our spectral data checked out, we need to next check if our P-T profile is aligned well with what our climate that we have run before says its i




First, we need to create our median band for our P-T profile alongside the 1-σ, 2-σ, 3-σ error from our Ultranest sampled points.

In [ ]:
#create band for p-t profile
def pt_band(params):
    final_goal = params[0:len(grid_parameters_unique.keys())]
    temp = lyz.custom_interp(final_goal, fitter, grid_name, to_interp='custom',
                                 array_to_interp=np.reshape(fitter.temperature['cldfree'],(3,6,8,4,91)))
    return temp
    


n_draws = 600
samples = results['samples']
draws=np.random.randint(0, samples.shape[0], n_draws)

pressure = np.reshape(fitter.pressure['cldfree'],(3,6,8,4,91))[0,0,0,0,:]
band = PredictionBand(pressure)#[6,4,1,1,1,:])
for idraw in draws:
    band.add(pt_band(samples[idraw,:]))

Next, we need to plot this band and overplot our climate model data to see if they are within some σ of each other.

In [ ]:
#plot it!
pressure = np.reshape(fitter.pressure['cldfree'],(3,6,8,4,91))[0,0,0,0,:]
fig, ax = plt.subplots(figsize=(7, 7))

band.line(color='k')
sigmas_hi = {}
sigmas_lo = {}
all_median={}

for q ,c,key in zip([k/100/2 for k in [68.27, 95.45, 99.73]], PuBuGn, ['1sig','2sig','3sig']): 
    #band.shade(q=q, color=c, alpha=0.5)
    sigmas_lo[key] = band.get_line(q=0.5 - q)
    all_median[key+'_lo_temperature'] = sigmas_lo[key]
    sigmas_hi[key] = band.get_line(q=0.5 + q)
    ax.fill_betweenx(pressure,sigmas_lo[key] ,sigmas_hi[key], color=c, alpha=0.5)
    all_median[key+'_hi_temperature'] = sigmas_hi[key]
median_best_fit = band.get_line(0.5)
all_median['temperature'] = median_best_fit
ax.plot(median_best_fit,pressure, color='black',alpha=1, label='median')
ax.plot(og_atmo['temperature'], og_atmo['pressure'], label='xarray', ls='--')

ax.set_ylabel('Pressure [bar]', fontsize=16)
ax.set_xlabel('Temperature [K]', fontsize=16)
# Set the font size of major tick labels on the x-axis and y-axis
ax.tick_params(axis='x', labelsize=13)
ax.tick_params(axis='y', labelsize=13)
ax.set_yscale('log')
ax.set_ylim([1e2,1e-6])
ax.set_xlim([800,3000])
ax.legend()

Looks dead on! This is great, our climate code is fully agreeing with the Ultranest sampled points. Just like the eclipse depth, the different shading represents 1-σ, 2-σ, 3-σ error. This is right down the middle which is fantastic.

# Chemical Analysis

This portion may be a bit more difficult considering we are not using transmission data, however we still can withdraw and dissect some really useful scientific data about the chemical state of the atmosphere.

Let's decide which molecules to analyze for.

In [ ]:
mols=['CO2','CH4','CO','H2O']

This is a very similar process as before. Create the chemical band, overplot with our climate data, and see the σ tolerance.

## Mixing Ratio

In [ ]:
chem = fitter.chemistry['cldfree']
#create our chemistry band for each chemical
def chem_band(params,mol):
    to_interp = np.reshape(chem[mol],(3,6,8,4,91))
    final_goal = params[0:len(grid_parameters_unique.keys())]
    temp = lyz.custom_interp(final_goal, fitter, grid_name, to_interp='custom',
                                       array_to_interp=to_interp )
    return temp
    

for imol in mols:
    n_draws = 200
    samples = results['samples']
    draws=np.random.randint(0, samples.shape[0], n_draws)

    band = PredictionBand(pressure)
    for idraw in draws:
        band.add(chem_band(samples[idraw,:], imol))
        
    median_best_fit = band.get_line(0.5)
    all_median[imol] = median_best_fit
    
    for q ,c,key in zip([k/100/2 for k in [68.27, 95.45, 99.73]], PuBuGn, ['1sig','2sig','3sig']): 
        sigmas_lo[key] = band.get_line(q=0.5 - q)
        all_median[key+'_lo_'+imol] = sigmas_lo[key]
        sigmas_hi[key] = band.get_line(q=0.5 + q)
        ax.fill_betweenx(pressure,sigmas_lo[key] ,sigmas_hi[key], color=c, alpha=0.5)
        all_median[key+'_hi_'+imol] = sigmas_hi[key]

In [ ]:
#plot it!
fig, ax = plt.subplots(figsize=(7, 7))
for imol in mols:
    ax.plot(all_median[imol],pressure,label=imol)
    ax.fill_betweenx(pressure,all_median['3sig'+'_lo_'+imol] ,
                     all_median['3sig'+'_hi_'+imol], alpha=0.5)

ax.plot(og_atmo['CO2'], og_atmo['pressure'], label='xarr CO2', ls="--")
ax.plot(og_atmo['CH4'], og_atmo['pressure'], label='xarr CH4', ls="--")
ax.plot(og_atmo['CO'], og_atmo['pressure'], label='xarr CO', ls="--")
ax.plot(og_atmo['H2O'], og_atmo['pressure'], label='xarr H2O', ls="--")
ax.set_ylabel('Pressure [bar]', fontsize=16)
ax.set_xlabel('(v/v)', fontsize=16)

ax.tick_params(axis='x', labelsize=13)
ax.tick_params(axis='y', labelsize=13)
ax.set_yscale('log')
ax.legend()
ax.set_xscale('log')
ax.set_ylim([1e2,1e-6])
ax.set_xlim([1e-7,1])

Since we don't have transmission data, our errors are much larger thus we only plot 3-σ shaded regions. But, it seems like our climate code and Ultranest sampled points are still in tandem!

Next, let's analyze which molecule is causing the biggest impact to our data by using the "leave-one-out" method. This can tell us which molecules are more than likely present than others. This may take a while as it effectively has to recreate the spectrum as many times as the molecules desired.

## "Leave One Out" Method (Molecular Contribution)

In [ ]:
import astropy.units as u

w,f,l =[],[],[]
for iex in mols + [None]:
    case.atmosphere(df = og_atmo, exclude_mol=iex, delim_whitespace=True)
    df= case.spectrum(opa, full_output=True,calculation='thermal')
    #print(df.keys())
    wno, rprs2  = df['wavenumber'] , df['fpfs_thermal']
    wno, rprs2 = jdi.mean_regrid(wno, rprs2, R=150)
    w +=[wno]
    f+=[rprs2]
    if iex==None:
        leg='all'
    else:
        leg = f'No {iex}'
    l+=[leg]
jpi.show(jpi.spectrum(w,f,legend=l))

This is awesome! We can clearly see and have a strong reasoning that H2O and CO2 are in the atmosphere. If they weren't present, the spectrum would completely veer off course of the data. 

# Thermal Contribution

Let's also look at the thermal contribution. This can give us the understanding of what fluxes are emitted at which pressure.

In [ ]:
df=case.spectrum(opa, full_output=True, calculation='emissions')
fig, ax, CF = jpi.thermal_contribution(df['full_output'],
                                       norm=jpi.colors.LogNorm(vmin=1e9, vmax=1e12))

# Building and Exporting Xarray

Now that our analysis of the sampled points is completed to our satisfaction, let's build and export an xarray of all the data used so anyone can make these plots and do any other analysis they desire. This is very important as you can effectively throw the whole model to someone and they can do whatever they want without having to rerun any modeling/sampling method




In [ ]:
import json 
from astropy.utils.misc import JsonCustomEncoder
import numpy.ma as ma

data_vars=dict(
        temperature_1sig_lo= (["pressure"], ma.getdata(all_median['1sig_lo_temperature']), {'units': 'Kelvin'}),
        temperature_1sig_hi= (["pressure"], ma.getdata(all_median['1sig_hi_temperature']), {'units': 'Kelvin'}),
        temperature_median= (["pressure"], ma.getdata(all_median['temperature']),{'units': 'Kelvin'}),
        eclipse_depth_median=(['wavelength'], spec_median_best_fit,{'units': 'Fp/F*'}),
        eclipse_depth_1sig_lo=(['wavelength'], ma.getdata(spec_sigmas_lo['1sig']),{'units': 'Fp/F*'}),
        eclipse_depth_1sig_hi=(['wavelength'], ma.getdata(spec_sigmas_hi['1sig']),{'units': 'Fp/F*'}),

    )

for i in mols:
    data_vars[i+"_median"] = (["pressure"], all_median[i])
    data_vars[i+"_1sig_lo"] = (["pressure"], all_median["1sig_lo_"+i])
    data_vars[i+"_1sig_hi"] = (["pressure"], all_median["1sig_hi_"+i])
build_xarray = xr.Dataset(
    data_vars=data_vars,
    coords=dict(
        pressure=(["pressure"], pressure, {'units': 'bar'}),
        wavelength=(['wavelength'], x_200, {'units': '1/cm'}),
    ),
    #change this with your information!
    attrs=dict(author="Dominic Doud",
               contact="dominic.doud@nasa.gov",
               model="PICASO Chemeq Grid + Virga fit with Ultranest",
               chisq_best_fit_perdata = chisq_best_fit_perdata,
               code="PICASO,Ultranest,Virga", #required, in this case I used numpy to make my fake model.
               median_params=json.dumps({ip:median_val[i] for i,ip in enumerate(paramnames)},cls=JsonCustomEncoder),
               summary=summary.to_json()
              ),
)

In [ ]:
build_xarray

In [ ]:
build_xarray.to_netcdf('final_picaso_cld_free_w_virga_medianfit.nc')